In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="D:\\temp_files\\datasets",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="D:\\temp_files\\datasets",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

超参数是可调整的参数，可用于控制模型优化过程。不同的超参数值会影响模型训练和收敛速度（阅读有关超参数调整的更多信息）

In [3]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

一旦我们设置了超参数，我们就可以通过优化循环来训练和优化我们的模型。优化循环的每次迭代都称为一个epoch。

常见的损失函数包括nn。回归任务的nn.MSELoss（均方误差）和分类任务nn.NLLLoss（负对数似然）。nn.CrossEntropyLoss组合nn.LogSoftmax和nn.NLLLoss。

In [ ]:
loss_fn = nn.CrossEntropyLoss()

优化算法定义了该过程的执行方式（在本例中，我们使用随机梯度下降）。所有优化逻辑都封装在优化器对象中。这里，我们使用SGD优化器；此外，PyTorch中还有许多不同的优化器，如ADAM和RMSProp，它们可以更好地处理不同类型的模型和数据。

1. 调用optimizer.zero_grad()重置模型参数的梯度。默认情况下，梯度相加；为了防止重复计数，我们在每次迭代时显式地将它们归零。
2. 通过调用loss.backward()反向传播预测损失。PyTorch存储了每个参数的损失梯度。
3. 一旦我们有了梯度，我们就调用optimizer.step()通过在反向过程中收集的梯度来调整参数。

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

我们定义了在优化代码上循环的train_lop和根据测试数据评估模型性能的test_lop。

In [10]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # 计算预测和损失
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # 反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [11]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
    print("Done!")

Epoch 1
-------------------------------
loss: 2.180351  [    0/60000]
loss: 2.169063  [ 6400/60000]
loss: 2.123678  [12800/60000]
loss: 2.132941  [19200/60000]
loss: 2.086291  [25600/60000]
loss: 2.033053  [32000/60000]
loss: 2.053170  [38400/60000]
loss: 1.984330  [44800/60000]
loss: 1.991882  [51200/60000]
loss: 1.917401  [57600/60000]
Test Error: 
 Accuracy: 55.4%, Avg loss: 1.921772 

Done!
Epoch 2
-------------------------------
loss: 1.957008  [    0/60000]
loss: 1.922169  [ 6400/60000]
loss: 1.821460  [12800/60000]
loss: 1.847377  [19200/60000]
loss: 1.742270  [25600/60000]
loss: 1.703351  [32000/60000]
loss: 1.703992  [38400/60000]
loss: 1.615712  [44800/60000]
loss: 1.643511  [51200/60000]
loss: 1.529242  [57600/60000]
Test Error: 
 Accuracy: 59.4%, Avg loss: 1.555586 

Done!
Epoch 3
-------------------------------
loss: 1.628713  [    0/60000]
loss: 1.583914  [ 6400/60000]
loss: 1.444339  [12800/60000]
loss: 1.495850  [19200/60000]
loss: 1.381937  [25600/60000]
loss: 1.388689